## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv",index_col=0)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Jamestown,US,42.0970,-79.2353,81.18,73,40,4.61,scattered clouds
1,Hobart,AU,-42.8794,147.3294,39.06,80,90,1.01,overcast clouds
2,Luwuk,ID,-0.9516,122.7875,76.51,89,100,3.27,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,73.85,80,5,9.01,clear sky
4,Faya,SA,18.3851,42.4509,72.12,58,72,10.27,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Choose minimum temperature"))
max_temp = float(input("Choose maximum temperature"))

Choose minimum temperature70
Choose maximum temperature100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & (city_data_df["Max Temp"]<= max_temp)]

#df["Max Temp"] >= min_temp
#city_data_df["Max Temp"]>= min_temp

#df["Max Temp"] <= max_temp
#city_data_df["Max Temp"]<= max_temp
df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Jamestown,US,42.0970,-79.2353,81.18,73,40,4.61,scattered clouds
2,Luwuk,ID,-0.9516,122.7875,76.51,89,100,3.27,overcast clouds
3,Rikitea,PF,-23.1203,-134.9692,73.85,80,5,9.01,clear sky
4,Faya,SA,18.3851,42.4509,72.12,58,72,10.27,broken clouds
5,Koungou,YT,-12.7336,45.2042,76.71,73,0,5.75,clear sky
...,...,...,...,...,...,...,...,...,...
668,Novoagansk,RU,61.9449,76.6625,70.27,67,91,4.05,overcast clouds
669,Marsabit,KE,2.3284,37.9899,75.88,38,9,17.16,clear sky
670,Emmen,NL,52.7792,6.9069,76.30,63,50,1.99,scattered clouds
671,Eyl,SO,7.9803,49.8164,78.48,71,91,26.78,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
(city_data_df).info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 675 entries, 0 to 674
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 675 non-null    object 
 1   Country              670 non-null    object 
 2   Lat                  675 non-null    float64
 3   Lng                  675 non-null    float64
 4   Max Temp             675 non-null    float64
 5   Humidity             675 non-null    int64  
 6   Cloudiness           675 non-null    int64  
 7   Wind Speed           675 non-null    float64
 8   Current Description  675 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 52.7+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df = city_data_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Jamestown,US,81.18,scattered clouds,42.0970,-79.2353,
1,Hobart,AU,39.06,overcast clouds,-42.8794,147.3294,
2,Luwuk,ID,76.51,overcast clouds,-0.9516,122.7875,
3,Rikitea,PF,73.85,clear sky,-23.1203,-134.9692,
4,Faya,SA,72.12,broken clouds,18.3851,42.4509,
5,Koungou,YT,76.71,clear sky,-12.7336,45.2042,
6,Khatanga,RU,51.58,clear sky,71.9667,102.5000,
7,Gornopravdinsk,RU,67.73,broken clouds,60.0500,69.9000,
8,Haukipudas,FI,76.14,clear sky,65.1765,25.3523,
9,Menongue,AO,84.24,clear sky,-14.6585,17.6910,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found")
    except:
        print("Hotel not found")
        

Hotel found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found


Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel not found
Hotel no

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]

In [10]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>city name</dt><dd>{City}</dd>
<dt>country code</dt><dd>{Country}</dd>
<dt>weather description</dt><dd>{Current Description}</dd>
<dt>maximum temperature</dt><dd>{Max Temp}</dd>

</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in city_data_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = city_data_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))